In [ ]:
import random
import math
import time
import matplotlib.pyplot as plt
from typing import List, Tuple, Dict, Optional

class City:
    """城市类，存储城市信息"""
    def __init__(self, id: int, x: float, y: float):
        self.id = id
        self.x = x
        self.y = y
    
    def distance_to(self, city) -> float:
        """计算到另一个城市的欧氏距离"""
        return math.sqrt((self.x - city.x) ** 2 + (self.y - city.y) ** 2)
    
    def __repr__(self):
        return f"City({self.id}: ({self.x}, {self.y}))"

class Individual:
    """个体类，代表一条可能的路径"""
    def __init__(self, path: List[int]):
        self.path = path  
        self.fitness = 0.0
        self.distance = 0.0
    
    def calculate_distance(self, distance_matrix: List[List[float]]) -> float:
        """计算路径总距离"""
        total_distance = 0.0
        n = len(self.path)
        
        for i in range(n):
            from_city = self.path[i]
            to_city = self.path[(i + 1) % n]  
            total_distance += distance_matrix[from_city][to_city]
        
        self.distance = total_distance
        return total_distance
    
    def calculate_fitness(self, distance_matrix: List[List[float]]) -> float:
        """计算适应度（路径越短适应度越高）"""
        if self.distance == 0:
            self.calculate_distance(distance_matrix)
        # 适应度 = 1 / 距离，距离越小适应度越大
        self.fitness = 1.0 / self.distance if self.distance > 0 else 0.0
        return self.fitness
    
    def __repr__(self):
        return f"Individual(path={self.path}, distance={self.distance:.2f})"

class GeneticAlgorithmTSP:
    """遗传算法求解TSP问题"""
    
    def __init__(self, cities: List[City], population_size: int = 100, 
                 generations: int = 500, crossover_rate: float = 0.8,
                 mutation_rate: float = 0.2, elite_size: int = 2,
                 tournament_size: int = 5):
        """
        初始化遗传算法参数
        
        参数:
            cities: 城市列表
            population_size: 种群大小
            generations: 迭代次数
            crossover_rate: 交叉概率
            mutation_rate: 变异概率
            elite_size: 精英个体数量（直接保留到下一代）
            tournament_size: 锦标赛选择的大小
        """
        self.cities = cities
        self.num_cities = len(cities)
        self.population_size = population_size
        self.generations = generations
        self.crossover_rate = crossover_rate
        self.mutation_rate = mutation_rate
        self.elite_size = elite_size
        self.tournament_size = tournament_size
        
        # 计算距离矩阵
        self.distance_matrix = self._create_distance_matrix()
        
        # 记录每代最佳值
        self.best_distances = []
        self.avg_distances = []
        self.worst_distances = []
        
        # 全局最优解
        self.best_individual = None
    
    def _create_distance_matrix(self) -> List[List[float]]:
        """创建距离矩阵"""
        n = self.num_cities
        distance_matrix = [[0.0] * n for _ in range(n)]
        
        for i in range(n):
            for j in range(n):
                if i != j:
                    distance_matrix[i][j] = self.cities[i].distance_to(self.cities[j])
        
        return distance_matrix
    
    def _create_random_individual(self) -> Individual:
        """创建随机个体（随机路径）"""
        # 创建0到n-1的列表，表示城市编号
        path = list(range(self.num_cities))
        # 随机打乱，但保证起点固定（可选）
        random.shuffle(path)
        return Individual(path)
    
    def _initialize_population(self) -> List[Individual]:
        """初始化种群"""
        population = []
        for _ in range(self.population_size):
            population.append(self._create_random_individual())
        
        # 计算初始适应度
        for ind in population:
            ind.calculate_fitness(self.distance_matrix)
        
        return population
    
    def _tournament_selection(self, population: List[Individual]) -> Individual:
        """锦标赛选择"""
        tournament = random.sample(population, self.tournament_size)
        # 选择适应度最高的个体
        return max(tournament, key=lambda x: x.fitness)
    
    def _order_crossover(self, parent1: Individual, parent2: Individual) -> Tuple[Individual, Individual]:
        """顺序交叉(Order Crossover, OX)"""
        n = self.num_cities
        
        # 选择两个随机交叉点
        point1 = random.randint(0, n - 1)
        point2 = random.randint(point1, n - 1)
        
        # 创建子代
        child1_path = [-1] * n
        child2_path = [-1] * n
        
        # 将交叉片段从parent1复制到child1，从parent2复制到child2
        for i in range(point1, point2 + 1):
            child1_path[i] = parent1.path[i]
            child2_path[i] = parent2.path[i]
        
        # 填充剩余位置
        self._fill_remaining_positions(child1_path, parent2.path, point1, point2)
        self._fill_remaining_positions(child2_path, parent1.path, point1, point2)
        
        return Individual(child1_path), Individual(child2_path)
    
    def _fill_remaining_positions(self, child_path: List[int], parent_path: List[int], 
                                  start: int, end: int):
        """填充剩余位置（用于顺序交叉）"""
        n = len(child_path)
        parent_idx = 0
        
        for i in range(n):
            # 找到要插入的位置
            insert_pos = (end + i + 1) % n
            
            # 如果这个位置已经有值，跳过
            if child_path[insert_pos] != -1:
                continue
            
            # 找到parent中不在孩子中的下一个城市
            while True:
                city = parent_path[parent_idx % n]
                parent_idx += 1
                
                if city not in child_path:
                    child_path[insert_pos] = city
                    break
    
    def _swap_mutation(self, individual: Individual) -> Individual:
        """交换变异：随机交换两个城市的位置"""
        if random.random() < self.mutation_rate:
            path = individual.path.copy()
            
            # 随机选择两个不同的位置
            idx1, idx2 = random.sample(range(len(path)), 2)
            
            # 交换
            path[idx1], path[idx2] = path[idx2], path[idx1]
            
            return Individual(path)
        
        return individual
    
    def _inversion_mutation(self, individual: Individual) -> Individual:
        """逆转变异：随机选择一段路径并反转"""
        if random.random() < self.mutation_rate:
            path = individual.path.copy()
            n = len(path)
            
            # 随机选择两个点
            point1 = random.randint(0, n - 1)
            point2 = random.randint(point1, n - 1)
            
            # 反转这段路径
            path[point1:point2+1] = reversed(path[point1:point2+1])
            
            return Individual(path)
        
        return individual
    
    def _scramble_mutation(self, individual: Individual) -> Individual:
        """打乱变异：随机选择一段路径并打乱"""
        if random.random() < self.mutation_rate:
            path = individual.path.copy()
            n = len(path)
            
            # 随机选择两个点
            point1 = random.randint(0, n - 1)
            point2 = random.randint(point1, n - 1)
            
            # 打乱这段路径
            segment = path[point1:point2+1]
            random.shuffle(segment)
            path[point1:point2+1] = segment
            
            return Individual(path)
        
        return individual
    
    def _calculate_population_stats(self, population: List[Individual]) -> Tuple[float, float, float]:
        """计算种群统计信息：最佳、平均、最差距离"""
        distances = [ind.distance for ind in population]
        return min(distances), sum(distances)/len(distances), max(distances)
    
    def evolve(self) -> Individual:
        """执行遗传算法进化"""
        # 初始化种群
        population = self._initialize_population()
        
        # 记录初始统计信息
        best_dist, avg_dist, worst_dist = self._calculate_population_stats(population)
        self.best_distances.append(best_dist)
        self.avg_distances.append(avg_dist)
        self.worst_distances.append(worst_dist)
        
        # 找到初始最佳个体
        self.best_individual = min(population, key=lambda x: x.distance)
        
        print(f"初始最佳距离: {best_dist:.2f}")
        
        # 开始进化
        for generation in range(self.generations):
            new_population = []
            
            # 精英选择：保留最佳个体
            population.sort(key=lambda x: x.fitness, reverse=True)
            new_population.extend(population[:self.elite_size])
            
            # 生成下一代
            while len(new_population) < self.population_size:
                # 选择父代
                parent1 = self._tournament_selection(population)
                parent2 = self._tournament_selection(population)
                
                # 交叉
                if random.random() < self.crossover_rate:
                    child1, child2 = self._order_crossover(parent1, parent2)
                else:
                    child1, child2 = parent1, parent2
                
                # 变异（可以选择不同的变异策略）
                child1 = self._swap_mutation(child1)
                child2 = self._swap_mutation(child2)
                
                # 计算适应度
                child1.calculate_fitness(self.distance_matrix)
                child2.calculate_fitness(self.distance_matrix)
                
                new_population.extend([child1, child2])
            
            # 更新种群
            population = new_population[:self.population_size]
            
            # 计算统计信息
            best_dist, avg_dist, worst_dist = self._calculate_population_stats(population)
            self.best_distances.append(best_dist)
            self.avg_distances.append(avg_dist)
            self.worst_distances.append(worst_dist)
            
            # 更新全局最佳个体
            current_best = min(population, key=lambda x: x.distance)
            if current_best.distance < self.best_individual.distance:
                self.best_individual = current_best
            
            # 每50代输出一次进度
            if (generation + 1) % 50 == 0:
                print(f"第 {generation + 1} 代: 最佳距离 = {best_dist:.2f}, 平均距离 = {avg_dist:.2f}")
        
        print(f"最终最佳距离: {self.best_individual.distance:.2f}")
        return self.best_individual
    
    def plot_convergence(self):
        """绘制收敛曲线"""
        plt.figure(figsize=(12, 6))
        
        plt.plot(self.best_distances, 'b-', label='最佳距离', linewidth=2)
        plt.plot(self.avg_distances, 'r-', label='平均距离', alpha=0.7)
        plt.plot(self.worst_distances, 'g-', label='最差距离', alpha=0.5)
        
        plt.xlabel('迭代次数')
        plt.ylabel('路径距离')
        plt.title('遗传算法收敛曲线')
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        # 添加水平线表示已知最优解（如果有的话）
        plt.axhline(y=5553, color='k', linestyle='--', alpha=0.5, label='已知最优解(5553)')
        
        plt.tight_layout()
        plt.show()
    
    def plot_route(self):
        """绘制最佳路径"""
        if not self.best_individual:
            print("没有找到最佳路径")
            return
        
        plt.figure(figsize=(10, 8))
        
        # 绘制城市
        x_coords = [city.x for city in self.cities]
        y_coords = [city.y for city in self.cities]
        
        plt.scatter(x_coords, y_coords, c='red', s=100, zorder=5)
        
        # 添加城市标签
        for i, city in enumerate(self.cities):
            plt.text(city.x, city.y, f'{i+1}', fontsize=8, ha='center', va='center', 
                    bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.7))
        
        # 绘制路径
        path = self.best_individual.path
        for i in range(len(path)):
            from_city = self.cities[path[i]]
            to_city = self.cities[path[(i + 1) % len(path)]]
            
            plt.plot([from_city.x, to_city.x], [from_city.y, to_city.y], 
                    'b-', alpha=0.6, linewidth=1)
        
        # 标记起点
        start_city = self.cities[path[0]]
        plt.scatter(start_city.x, start_city.y, c='green', s=150, 
                   marker='*', zorder=10, label='起点')
        
        plt.xlabel('X坐标')
        plt.ylabel('Y坐标')
        plt.title(f'TSP最优路径 (总距离: {self.best_individual.distance:.2f})')
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()

def load_cities_from_coordinates(coordinates: List[Tuple[int, float, float]]) -> List[City]:
    """从坐标数据加载城市"""
    cities = []
    for city_id, x, y in coordinates:
        # 注意：城市编号在内部从0开始，但显示时从1开始
        cities.append(City(city_id - 1, x, y))
    return cities

def main():
    """主函数"""
    # 定义城市坐标数据（50个城市）
    coordinates = [
        (1, 475, 110), (2, 29, 929), (3, 755, 897), (4, 698, 979), (5, 871, 317),
        (6, 535, 184), (7, 548, 73), (8, 594, 380), (9, 233, 845), (10, 881, 654),
        (11, 200, 540), (12, 9, 800), (13, 526, 456), (14, 765, 30), (15, 992, 449),
        (16, 659, 193), (17, 938, 370), (18, 233, 966), (19, 692, 112), (20, 680, 873),
        (21, 589, 968), (22, 553, 444), (23, 106, 941), (24, 959, 366), (25, 273, 614),
        (26, 568, 383), (27, 661, 431), (28, 824, 144), (29, 191, 587), (30, 464, 457),
        (31, 959, 722), (32, 823, 486), (33, 259, 513), (34, 977, 467), (35, 794, 895),
        (36, 243, 863), (37, 802, 294), (38, 811, 650), (39, 954, 534), (40, 300, 486),
        (41, 665, 865), (42, 648, 205), (43, 645, 848), (44, 29, 496), (45, 98, 25),
        (46, 344, 676), (47, 735, 371), (48, 924, 450), (49, 313, 472), (50, 407, 339)
    ]
    
    print("=" * 60)
    print("基于遗传算法的TSP问题求解器")
    print("=" * 60)
    
    # 加载城市数据
    cities = load_cities_from_coordinates(coordinates)
    print(f"已加载 {len(cities)} 个城市")
    
    # 参数设置
    population_size = 150  # 种群大小
    generations = 800      # 迭代次数
    crossover_rate = 0.85  # 交叉概率
    mutation_rate = 0.15   # 变异概率
    
    print("\n算法参数:")
    print(f"  种群大小: {population_size}")
    print(f"  迭代次数: {generations}")
    print(f"  交叉概率: {crossover_rate}")
    print(f"  变异概率: {mutation_rate}")
    
    # 创建遗传算法实例
    start_time = time.time()
    
    ga_tsp = GeneticAlgorithmTSP(
        cities=cities,
        population_size=population_size,
        generations=generations,
        crossover_rate=crossover_rate,
        mutation_rate=mutation_rate,
        elite_size=5,
        tournament_size=10
    )
    
    # 运行算法
    print("\n开始运行遗传算法...")
    best_solution = ga_tsp.evolve()
    
    end_time = time.time()
    execution_time = end_time - start_time
    
    print("\n" + "=" * 60)
    print("求解结果:")
    print("=" * 60)
    
    print(f"最佳路径长度: {best_solution.distance:.2f}")
    print(f"与已知最优解(5553)的差距: {abs(best_solution.distance - 5553):.2f}")
    print(f"差距百分比: {abs(best_solution.distance - 5553) / 5553 * 100:.2f}%")
    print(f"算法运行时间: {execution_time:.2f}秒")
    
    # 显示最佳路径（城市编号从1开始）
    print("\n最佳路径（城市访问顺序）:")
    path_display = [city_id + 1 for city_id in best_solution.path]
    # 每行显示10个城市
    for i in range(0, len(path_display), 10):
        print("  ", path_display[i:i+10])
    
    # 绘制结果
    print("\n正在生成可视化结果...")
    ga_tsp.plot_convergence()
    ga_tsp.plot_route()
    
    # 分析不同参数的影响
    print("\n" + "=" * 60)
    print("参数敏感性分析（示例）:")
    print("=" * 60)
    
    # 测试不同种群大小
    test_pop_sizes = [50, 100, 150, 200]
    results_pop = []
    
    for pop_size in test_pop_sizes:
        ga_test = GeneticAlgorithmTSP(
            cities=cities,
            population_size=pop_size,
            generations=300,  # 为了快速测试，减少迭代次数
            crossover_rate=0.8,
            mutation_rate=0.15
        )
        best = ga_test.evolve()
        results_pop.append((pop_size, best.distance))
    
    print("\n不同种群大小的影响:")
    for pop_size, distance in results_pop:
        print(f"  种群大小 {pop_size}: 最佳距离 = {distance:.2f}")
    
    # 测试不同变异概率
    test_mutation_rates = [0.05, 0.1, 0.15, 0.2, 0.3]
    results_mutation = []
    
    for mut_rate in test_mutation_rates:
        ga_test = GeneticAlgorithmTSP(
            cities=cities,
            population_size=100,
            generations=300,
            crossover_rate=0.8,
            mutation_rate=mut_rate
        )
        best = ga_test.evolve()
        results_mutation.append((mut_rate, best.distance))
    
    print("\n不同变异概率的影响:")
    for mut_rate, distance in results_mutation:
        print(f"  变异概率 {mut_rate}: 最佳距离 = {distance:.2f}")
    
    print("\n" + "=" * 60)
    print("算法总结:")
    print("=" * 60)
    print("1. 遗传算法能有效求解TSP问题")
    print("2. 种群大小和迭代次数对结果质量有显著影响")
    print("3. 变异概率需要适中，太高会破坏好解，太低会陷入局部最优")
    print("4. 精英保留策略有助于保持种群多样性")
    print("5. 顺序交叉(OX)特别适合TSP问题")
    print(f"\n本次求解的最佳距离为: {best_solution.distance:.2f}")

if __name__ == "__main__":
    # 设置随机种子以获得可重复的结果
    random.seed(42)
    main()

In [ ]:
import random
import math
import time
import matplotlib.pyplot as plt
from typing import List, Tuple, Dict, Optional
from tqdm import tqdm  # 导入进度条库

class City:
    """城市类，存储城市信息"""
    def __init__(self, id: int, x: float, y: float):
        self.id = id
        self.x = x
        self.y = y
    
    def distance_to(self, city) -> float:
        """计算到另一个城市的欧氏距离"""
        return math.sqrt((self.x - city.x) ** 2 + (self.y - city.y) ** 2)
    
    def __repr__(self):
        return f"City({self.id}: ({self.x}, {self.y}))"

class Individual:
    """个体类，代表一条可能的路径"""
    def __init__(self, path: List[int]):
        self.path = path  
        self.fitness = 0.0
        self.distance = 0.0
    
    def calculate_distance(self, distance_matrix: List[List[float]]) -> float:
        """计算路径总距离"""
        total_distance = 0.0
        n = len(self.path)
        
        for i in range(n):
            from_city = self.path[i]
            to_city = self.path[(i + 1) % n]  
            total_distance += distance_matrix[from_city][to_city]
        
        self.distance = total_distance
        return total_distance
    
    def calculate_fitness(self, distance_matrix: List[List[float]]) -> float:
        """计算适应度（路径越短适应度越高）"""
        if self.distance == 0:
            self.calculate_distance(distance_matrix)
        # 适应度 = 1 / 距离，距离越小适应度越大
        self.fitness = 1.0 / self.distance if self.distance > 0 else 0.0
        return self.fitness
    
    def __repr__(self):
        return f"Individual(path={self.path}, distance={self.distance:.2f})"

class GeneticAlgorithmTSP:
    """遗传算法求解TSP问题"""
    
    def __init__(self, cities: List[City], population_size: int = 100, 
                 generations: int = 500, crossover_rate: float = 0.8,
                 mutation_rate: float = 0.2, elite_size: int = 2,
                 tournament_size: int = 5):
        """
        初始化遗传算法参数
        
        参数:
            cities: 城市列表
            population_size: 种群大小
            generations: 迭代次数
            crossover_rate: 交叉概率
            mutation_rate: 变异概率
            elite_size: 精英个体数量（直接保留到下一代）
            tournament_size: 锦标赛选择的大小
        """
        self.cities = cities
        self.num_cities = len(cities)
        self.population_size = population_size
        self.generations = generations
        self.crossover_rate = crossover_rate
        self.mutation_rate = mutation_rate
        self.elite_size = elite_size
        self.tournament_size = tournament_size
        
        # 计算距离矩阵
        self.distance_matrix = self._create_distance_matrix()
        
        # 记录每代最佳值
        self.best_distances = []
        self.avg_distances = []
        self.worst_distances = []
        
        # 全局最优解
        self.best_individual = None
    
    def _create_distance_matrix(self) -> List[List[float]]:
        """创建距离矩阵"""
        n = self.num_cities
        distance_matrix = [[0.0] * n for _ in range(n)]
        
        for i in range(n):
            for j in range(n):
                if i != j:
                    distance_matrix[i][j] = self.cities[i].distance_to(self.cities[j])
        
        return distance_matrix
    
    def _create_random_individual(self) -> Individual:
        """创建随机个体（随机路径）"""
        # 创建0到n-1的列表，表示城市编号
        path = list(range(self.num_cities))
        # 随机打乱，但保证起点固定（可选）
        random.shuffle(path)
        return Individual(path)
    
    def _initialize_population(self) -> List[Individual]:
        """初始化种群"""
        population = []
        for _ in range(self.population_size):
            population.append(self._create_random_individual())
        
        # 计算初始适应度
        for ind in population:
            ind.calculate_fitness(self.distance_matrix)
        
        return population
    
    def _tournament_selection(self, population: List[Individual]) -> Individual:
        """锦标赛选择"""
        tournament = random.sample(population, self.tournament_size)
        # 选择适应度最高的个体
        return max(tournament, key=lambda x: x.fitness)
    
    def _order_crossover(self, parent1: Individual, parent2: Individual) -> Tuple[Individual, Individual]:
        """顺序交叉(Order Crossover, OX)"""
        n = self.num_cities
        
        # 选择两个随机交叉点
        point1 = random.randint(0, n - 1)
        point2 = random.randint(point1, n - 1)
        
        # 创建子代
        child1_path = [-1] * n
        child2_path = [-1] * n
        
        # 将交叉片段从parent1复制到child1，从parent2复制到child2
        for i in range(point1, point2 + 1):
            child1_path[i] = parent1.path[i]
            child2_path[i] = parent2.path[i]
        
        # 填充剩余位置
        self._fill_remaining_positions(child1_path, parent2.path, point1, point2)
        self._fill_remaining_positions(child2_path, parent1.path, point1, point2)
        
        return Individual(child1_path), Individual(child2_path)
    
    def _fill_remaining_positions(self, child_path: List[int], parent_path: List[int], 
                                  start: int, end: int):
        """填充剩余位置（用于顺序交叉）"""
        n = len(child_path)
        parent_idx = 0
        
        for i in range(n):
            # 找到要插入的位置
            insert_pos = (end + i + 1) % n
            
            # 如果这个位置已经有值，跳过
            if child_path[insert_pos] != -1:
                continue
            
            # 找到parent中不在孩子中的下一个城市
            while True:
                city = parent_path[parent_idx % n]
                parent_idx += 1
                
                if city not in child_path:
                    child_path[insert_pos] = city
                    break
    
    def _swap_mutation(self, individual: Individual) -> Individual:
        """交换变异：随机交换两个城市的位置"""
        if random.random() < self.mutation_rate:
            path = individual.path.copy()
            
            # 随机选择两个不同的位置
            idx1, idx2 = random.sample(range(len(path)), 2)
            
            # 交换
            path[idx1], path[idx2] = path[idx2], path[idx1]
            
            return Individual(path)
        
        return individual
    
    def _inversion_mutation(self, individual: Individual) -> Individual:
        """逆转变异：随机选择一段路径并反转"""
        if random.random() < self.mutation_rate:
            path = individual.path.copy()
            n = len(path)
            
            # 随机选择两个点
            point1 = random.randint(0, n - 1)
            point2 = random.randint(point1, n - 1)
            
            # 反转这段路径
            path[point1:point2+1] = reversed(path[point1:point2+1])
            
            return Individual(path)
        
        return individual
    
    def _scramble_mutation(self, individual: Individual) -> Individual:
        """打乱变异：随机选择一段路径并打乱"""
        if random.random() < self.mutation_rate:
            path = individual.path.copy()
            n = len(path)
            
            # 随机选择两个点
            point1 = random.randint(0, n - 1)
            point2 = random.randint(point1, n - 1)
            
            # 打乱这段路径
            segment = path[point1:point2+1]
            random.shuffle(segment)
            path[point1:point2+1] = segment
            
            return Individual(path)
        
        return individual
    
    def _calculate_population_stats(self, population: List[Individual]) -> Tuple[float, float, float]:
        """计算种群统计信息：最佳、平均、最差距离"""
        distances = [ind.distance for ind in population]
        return min(distances), sum(distances)/len(distances), max(distances)
    
    def evolve(self) -> Individual:
        """执行遗传算法进化"""
        # 初始化种群
        population = self._initialize_population()
        
        # 记录初始统计信息
        best_dist, avg_dist, worst_dist = self._calculate_population_stats(population)
        self.best_distances.append(best_dist)
        self.avg_distances.append(avg_dist)
        self.worst_distances.append(worst_dist)
        
        # 找到初始最佳个体
        self.best_individual = min(population, key=lambda x: x.distance)
        
        print(f"初始最佳距离: {best_dist:.2f}")
        
        # 开始进化，添加进度条
        # 使用tqdm创建进度条，desc设置描述，total设置总步数
        progress_bar = tqdm(range(self.generations), desc="遗传算法迭代", unit="代")
        
        for generation in progress_bar:
            new_population = []
            
            # 精英选择：保留最佳个体
            population.sort(key=lambda x: x.fitness, reverse=True)
            new_population.extend(population[:self.elite_size])
            
            # 生成下一代
            while len(new_population) < self.population_size:
                # 选择父代
                parent1 = self._tournament_selection(population)
                parent2 = self._tournament_selection(population)
                
                # 交叉
                if random.random() < self.crossover_rate:
                    child1, child2 = self._order_crossover(parent1, parent2)
                else:
                    child1, child2 = parent1, parent2
                
                # 变异（可以选择不同的变异策略）
                child1 = self._swap_mutation(child1)
                child2 = self._swap_mutation(child2)
                
                # 计算适应度
                child1.calculate_fitness(self.distance_matrix)
                child2.calculate_fitness(self.distance_matrix)
                
                new_population.extend([child1, child2])
            
            # 更新种群
            population = new_population[:self.population_size]
            
            # 计算统计信息
            best_dist, avg_dist, worst_dist = self._calculate_population_stats(population)
            self.best_distances.append(best_dist)
            self.avg_distances.append(avg_dist)
            self.worst_distances.append(worst_dist)
            
            # 更新全局最佳个体
            current_best = min(population, key=lambda x: x.distance)
            if current_best.distance < self.best_individual.distance:
                self.best_individual = current_best
            
            # 更新进度条的显示信息
            progress_bar.set_postfix({
                '当前最佳距离': f'{best_dist:.2f}',
                '全局最佳距离': f'{self.best_individual.distance:.2f}',
                '平均距离': f'{avg_dist:.2f}'
            })
            
            # 每50代输出一次详细进度（可选）
            if (generation + 1) % 50 == 0:
                tqdm.write(f"第 {generation + 1} 代: 最佳距离 = {best_dist:.2f}, 平均距离 = {avg_dist:.2f}")
        
        # 关闭进度条
        progress_bar.close()
        
        print(f"最终最佳距离: {self.best_individual.distance:.2f}")
        return self.best_individual
    
    def plot_convergence(self):
        """绘制收敛曲线"""
        plt.figure(figsize=(12, 6))
        
        plt.plot(self.best_distances, 'b-', label='最佳距离', linewidth=2)
        plt.plot(self.avg_distances, 'r-', label='平均距离', alpha=0.7)
        plt.plot(self.worst_distances, 'g-', label='最差距离', alpha=0.5)
        
        plt.xlabel('迭代次数')
        plt.ylabel('路径距离')
        plt.title('遗传算法收敛曲线')
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        # 添加水平线表示已知最优解（如果有的话）
        plt.axhline(y=5553, color='k', linestyle='--', alpha=0.5, label='已知最优解(5553)')
        
        plt.tight_layout()
        plt.show()
    
    def plot_route(self):
        """绘制最佳路径"""
        if not self.best_individual:
            print("没有找到最佳路径")
            return
        
        plt.figure(figsize=(10, 8))
        
        # 绘制城市
        x_coords = [city.x for city in self.cities]
        y_coords = [city.y for city in self.cities]
        
        plt.scatter(x_coords, y_coords, c='red', s=100, zorder=5)
        
        # 添加城市标签
        for i, city in enumerate(self.cities):
            plt.text(city.x, city.y, f'{i+1}', fontsize=8, ha='center', va='center', 
                    bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.7))
        
        # 绘制路径
        path = self.best_individual.path
        for i in range(len(path)):
            from_city = self.cities[path[i]]
            to_city = self.cities[path[(i + 1) % len(path)]]
            
            plt.plot([from_city.x, to_city.x], [from_city.y, to_city.y], 
                    'b-', alpha=0.6, linewidth=1)
        
        # 标记起点
        start_city = self.cities[path[0]]
        plt.scatter(start_city.x, start_city.y, c='green', s=150, 
                   marker='*', zorder=10, label='起点')
        
        plt.xlabel('X坐标')
        plt.ylabel('Y坐标')
        plt.title(f'TSP最优路径 (总距离: {self.best_individual.distance:.2f})')
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()

def load_cities_from_coordinates(coordinates: List[Tuple[int, float, float]]) -> List[City]:
    """从坐标数据加载城市"""
    cities = []
    for city_id, x, y in coordinates:
        # 注意：城市编号在内部从0开始，但显示时从1开始
        cities.append(City(city_id - 1, x, y))
    return cities

def main():
    """主函数"""
    # 定义城市坐标数据（50个城市）
    coordinates = [
        (1, 475, 110), (2, 29, 929), (3, 755, 897), (4, 698, 979), (5, 871, 317),
        (6, 535, 184), (7, 548, 73), (8, 594, 380), (9, 233, 845), (10, 881, 654),
        (11, 200, 540), (12, 9, 800), (13, 526, 456), (14, 765, 30), (15, 992, 449),
        (16, 659, 193), (17, 938, 370), (18, 233, 966), (19, 692, 112), (20, 680, 873),
        (21, 589, 968), (22, 553, 444), (23, 106, 941), (24, 959, 366), (25, 273, 614),
        (26, 568, 383), (27, 661, 431), (28, 824, 144), (29, 191, 587), (30, 464, 457),
        (31, 959, 722), (32, 823, 486), (33, 259, 513), (34, 977, 467), (35, 794, 895),
        (36, 243, 863), (37, 802, 294), (38, 811, 650), (39, 954, 534), (40, 300, 486),
        (41, 665, 865), (42, 648, 205), (43, 645, 848), (44, 29, 496), (45, 98, 25),
        (46, 344, 676), (47, 735, 371), (48, 924, 450), (49, 313, 472), (50, 407, 339)
    ]
    
    print("=" * 60)
    print("基于遗传算法的TSP问题求解器")
    print("=" * 60)
    
    # 加载城市数据
    cities = load_cities_from_coordinates(coordinates)
    print(f"已加载 {len(cities)} 个城市")
    
    # 参数设置
    population_size = 150  # 种群大小
    generations = 800      # 迭代次数
    crossover_rate = 0.85  # 交叉概率
    mutation_rate = 0.15   # 变异概率
    
    print("\n算法参数:")
    print(f"  种群大小: {population_size}")
    print(f"  迭代次数: {generations}")
    print(f"  交叉概率: {crossover_rate}")
    print(f"  变异概率: {mutation_rate}")
    
    # 创建遗传算法实例
    start_time = time.time()
    
    ga_tsp = GeneticAlgorithmTSP(
        cities=cities,
        population_size=population_size,
        generations=generations,
        crossover_rate=crossover_rate,
        mutation_rate=mutation_rate,
        elite_size=5,
        tournament_size=10
    )
    
    # 运行算法
    print("\n开始运行遗传算法...")
    best_solution = ga_tsp.evolve()
    
    end_time = time.time()
    execution_time = end_time - start_time
    
    print("\n" + "=" * 60)
    print("求解结果:")
    print("=" * 60)
    
    print(f"最佳路径长度: {best_solution.distance:.2f}")
    print(f"与已知最优解(5553)的差距: {abs(best_solution.distance - 5553):.2f}")
    print(f"差距百分比: {abs(best_solution.distance - 5553) / 5553 * 100:.2f}%")
    print(f"算法运行时间: {execution_time:.2f}秒")
    
    # 显示最佳路径（城市编号从1开始）
    print("\n最佳路径（城市访问顺序）:")
    path_display = [city_id + 1 for city_id in best_solution.path]
    # 每行显示10个城市
    for i in range(0, len(path_display), 10):
        print("  ", path_display[i:i+10])
    
    # 绘制结果
    print("\n正在生成可视化结果...")
    ga_tsp.plot_convergence()
    ga_tsp.plot_route()
    
    # 分析不同参数的影响
    print("\n" + "=" * 60)
    print("参数敏感性分析（示例）:")
    print("=" * 60)
    
    # 测试不同种群大小（添加进度提示）
    test_pop_sizes = [50, 100, 150, 200]
    results_pop = []
    
    print("\n测试不同种群大小的影响...")
    for pop_size in tqdm(test_pop_sizes, desc="种群大小测试"):
        ga_test = GeneticAlgorithmTSP(
            cities=cities,
            population_size=pop_size,
            generations=300,  # 为了快速测试，减少迭代次数
            crossover_rate=0.8,
            mutation_rate=0.15
        )
        best = ga_test.evolve()
        results_pop.append((pop_size, best.distance))
    
    print("\n不同种群大小的影响:")
    for pop_size, distance in results_pop:
        print(f"  种群大小 {pop_size}: 最佳距离 = {distance:.2f}")
    
    # 测试不同变异概率（添加进度提示）
    test_mutation_rates = [0.05, 0.1, 0.15, 0.2, 0.3]
    results_mutation = []
    
    print("\n测试不同变异概率的影响...")
    for mut_rate in tqdm(test_mutation_rates, desc="变异概率测试"):
        ga_test = GeneticAlgorithmTSP(
            cities=cities,
            population_size=100,
            generations=300,
            crossover_rate=0.8,
            mutation_rate=mut_rate
        )
        best = ga_test.evolve()
        results_mutation.append((mut_rate, best.distance))
    
    print("\n不同变异概率的影响:")
    for mut_rate, distance in results_mutation:
        print(f"  变异概率 {mut_rate}: 最佳距离 = {distance:.2f}")
    
    print("\n" + "=" * 60)
    print("算法总结:")
    print("=" * 60)
    print("1. 遗传算法能有效求解TSP问题")
    print("2. 种群大小和迭代次数对结果质量有显著影响")
    print("3. 变异概率需要适中，太高会破坏好解，太低会陷入局部最优")
    print("4. 精英保留策略有助于保持种群多样性")
    print("5. 顺序交叉(OX)特别适合TSP问题")
    print(f"\n本次求解的最佳距离为: {best_solution.distance:.2f}")

if __name__ == "__main__":
    # 设置随机种子以获得可重复的结果
    random.seed(42)
    # 安装tqdm（如果未安装）
    try:
        import tqdm
    except ImportError:
        import subprocess
        import sys
        subprocess.check_call([sys.executable, "-m", "pip", "install", "tqdm"])
        import tqdm
    
    main()